In [1]:
import torch
import numpy as np
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from transformers import BertModel 

# 日本語トークナイザ
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese')
# 事前学習済みBert
model = BertModel.from_pretrained('bert-base-japanese')

In [2]:
def cos(x1, x2):
    """
    引数の２つベクトルの内積を計算する
    """
    return np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2))

def create_vec(text):
    """ 入力されたテキストを推論する """
    input_batch = [text]

    # トークナイズでテキストを分割する
    encoded_data = tokenizer.batch_encode_plus(
        input_batch, pad_to_max_length=True, add_special_tokens=True)

    # 単語ベクトルをIDに変換する
    input_ids = torch.tensor(encoded_data["input_ids"])

    # 推論
    with torch.no_grad():
        outputs = model(input_ids)

    # 最終層の情報と、分割したトークンを返す
    last_hidden_states = outputs[0]
    return last_hidden_states.detach().numpy(), tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

In [3]:
%time
text = "私は明日から自宅勤務です。腰を傷めないようにお高い椅子を買おうと思います"
last_hidden_states, token = create_vec(text)
print(last_hidden_states.shape)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs
(1, 27, 768)


# 文ベクトルを比較してみる

In [4]:
text1 = "IT企業ではないアマゾンにはよく釣りに行く"
text2 = "南米のアマゾン川は危険な生き物がたくさんいる"
text3 = "アマゾンで椅子を買ってリモートワーク環境を作る"

In [5]:
vec1, _ = create_vec(text1)
vec2, _ = create_vec(text2)
vec3, _ = create_vec(text3)

# 文ベクトルの類似度を出力
print("「{}...」と「{}...」の文ベクトル類似度：{:.5f}".format(text1[:5], text2[:5], cos(vec1[0, 0, :], vec2[0, 0, :])))
print("「{}...」と「{}...」の文ベクトル類似度：{:.5f}".format(text1[:5], text3[:5], cos(vec1[0, 0, :], vec3[0, 0, :])))
print("「{}...」と「{}...」の文ベクトル類似度：{:.5f}".format(text3[:5], text2[:5], cos(vec3[0, 0, :], vec2[0, 0, :])))

「IT企業で...」と「南米のアマ...」の文ベクトル類似度：0.72820
「IT企業で...」と「アマゾンで...」の文ベクトル類似度：0.71819
「アマゾンで...」と「南米のアマ...」の文ベクトル類似度：0.68089


## 全然関係ない文章同士では？

In [6]:
text1 = "アマゾンで椅子を買ってリモートワーク環境を作る"
text2 = "アマゾンのクラウドを利用してシステムを構築する"
except_text = "明日の晩御飯は豚の生姜焼きともやし炒めにする"

In [7]:
vec1, _ = create_vec(text1)
vec2, _ = create_vec(text2)
ex_vec, _ = create_vec(except_text)

# 文ベクトルの類似度を出力
print("「{}...」と「{}...」の文ベクトル類似度：{:.5f}".format(text1[:5], text2[:5], cos(vec1[0, 0, :], vec2[0, 0, :])))
print("「{}...」と「{}...」の文ベクトル類似度：{:.5f}".format(text1[:5], except_text[:5], cos(vec1[0, 0, :], ex_vec[0, 0, :])))
print("「{}...」と「{}...」の文ベクトル類似度：{:.5f}".format(except_text[:5], text2[:5], cos(ex_vec[0, 0, :], vec2[0, 0, :])))

「アマゾンで...」と「アマゾンの...」の文ベクトル類似度：0.81488
「アマゾンで...」と「明日の晩御...」の文ベクトル類似度：0.67456
「明日の晩御...」と「アマゾンの...」の文ベクトル類似度：0.65609


# 特定単語ベクトルを比較してみる

In [8]:
text1 = "IT企業ではないアマゾンにはよく釣りに行く"
text2 = "南米のアマゾン川は危険な生き物がたくさんいる"
text3 = "アマゾンで椅子を買ってリモートワーク環境を作る"

In [9]:
vec1, token1 = create_vec(text1)
vec2, token2 = create_vec(text2)
vec3, token3 = create_vec(text3)

# 対象の単語
TARGET = "アマゾン"

# 単語ベクトルの類似度を出力
# TARGETのインデックスを取得し、それに対応するベクトルで計算する
print("「{}...」と「{}...」の文ベクトル類似度：{:.5f}".format(text1[:5], text2[:5], cos(vec1[0, token1.index(TARGET), :], vec2[0, token2.index(TARGET), :])))
print("「{}...」と「{}...」の文ベクトル類似度：{:.5f}".format(text1[:5], text3[:5], cos(vec1[0, token1.index(TARGET), :], vec3[0, token3.index(TARGET), :])))
print("「{}...」と「{}...」の文ベクトル類似度：{:.5f}".format(text3[:5], text2[:5], cos(vec3[0, token3.index(TARGET), :], vec2[0, token2.index(TARGET), :])))

「IT企業で...」と「南米のアマ...」の文ベクトル類似度：0.65961
「IT企業で...」と「アマゾンで...」の文ベクトル類似度：0.71137
「アマゾンで...」と「南米のアマ...」の文ベクトル類似度：0.68076
